In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [23]:
train_data = datasets.CIFAR100(root='./dir1', train=True, download=True, transform=transform)
test_data = datasets.CIFAR100(root='./dir1', train=False, download=True, transform=transform)

100.0%


In [24]:
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

In [25]:
class cnn(nn.Module):
    def __init__(self):
        super(cnn, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        
        self.fc1 = nn.Linear(8 * 8 * 64, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 100)
        # self.relu = nn.ReLU()
        # self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool1(torch.relu(x))
        x = self.conv2(x)
        x = self.pool2(torch.relu(x))
        x = x.flatten(1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [26]:
model = cnn()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [27]:
epochs = 10
for epoch in range(epochs):
    for image, label in train_loader:
        output = model(image)
        loss = criterion(output, label) 
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

Epoch 1, Loss: 3.2230591773986816
Epoch 2, Loss: 2.4250643253326416
Epoch 3, Loss: 2.4544689655303955
Epoch 4, Loss: 2.2508134841918945
Epoch 5, Loss: 2.011507272720337
Epoch 6, Loss: 2.4985265731811523
Epoch 7, Loss: 2.163684606552124
Epoch 8, Loss: 1.2470712661743164
Epoch 9, Loss: 1.1436223983764648
Epoch 10, Loss: 1.7961065769195557


In [28]:
correct = 0
l=[]
model.eval()
with torch.no_grad():
    for image, label in test_loader:
        output = model(image)
        max, predicted = torch.max(output, 1)
        correct += (predicted == label).sum().item()
        accuracy = correct / len(test_data)
        l.append(accuracy)
print(f'Test Accuracy: {(sum(l)/len(l)) * 100:.2f}%')

Test Accuracy: 19.31%
